In [1]:
%pip install python-dotenv

## 环境变量

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(os.environ)

AI_TEMPERATURE=0.0
AI_MAX_TOKENS=2048

## 模型加载

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

def load_azureLLM():
    # chat
    chat = AzureChatOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_CHAT'),
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    # llm
    llm = AzureOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_LLM'),
        model_name="text-davinci-003",
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    
    embedding = OpenAIEmbeddings(deployment = os.environ.get('DEPLOYMENT_NAME_EMBEDDING'),chunk_size=1)
    
    return llm,chat,embedding

llm,chat,embed = load_azureLLM()

In [4]:
import asyncio
import time

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


### 顺序请求

In [5]:

def generate_serially():
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    for _ in range(5):
        resp = chain.run(product="toothpaste")
        print(resp)


In [6]:

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print("\033[1m" + f"顺序请求 executed in {elapsed:0.2f} seconds." + "\033[0m")



BrightSmile Toothpaste Co.


BrightSmile Toothpaste Co.


BrightSmile Toothpaste Co.


BrightSmile Toothpaste Co.


BrightSmile Toothpaste Co.
顺序请求 executed in 5.44 seconds.


### 并发请求

In [7]:

async def async_generate(chain):
    resp = await chain.arun(product="toothpaste")
    print(resp)

async def generate_concurrently():
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tasks = [async_generate(chain) for _ in range(5)]
    await asyncio.gather(*tasks)


In [8]:

s = time.perf_counter()
# If running this outside of Jupyter, use asyncio.run(generate_concurrently())
await generate_concurrently()
elapsed = time.perf_counter() - s
print("\033[1m" + f"并发请求 executed in {elapsed:0.2f} seconds." + "\033[0m")




BrightSmile Toothpaste Co.


BrightSmile Toothpaste Co.


BrightSmile Toothpaste Co.


BrightSmile Toothpaste Co.


BrightSmile Toothpaste Co.
并发请求 executed in 1.24 seconds.
